# 안녕하세요^^ 
# AIVLE 미니 프로젝트에 오신 여러분을 환영합니다.
* 본 과정에서는 실제 사례와 데이터를 기반으로 문제를 해결하는 전체 과정을 자기 주도형 실습으로 진행해볼 예정입니다.
* 앞선 교육과정을 정리하는 마음과 지금까지 배운 내용을 바탕으로 문제 해결을 해볼게요!
* 미니 프로젝트를 통한 문제 해결 과정 'A에서 Z까지', 지금부터 시작합니다!


### Text Classification
> * [Google Tutorial](https://developers.google.com/machine-learning/guides/text-classification)
> * [Tensorflow Tutorial](https://www.tensorflow.org/tutorials/keras/text_classification)
> * [Keras-tutorial](https://keras.io/examples/nlp/text_classification_from_scratch/)
> * [BERT-tutorial](https://www.tensorflow.org/text/guide/bert_preprocessing_guide)

---
#### 

### 0. library

In [1]:
# 필요 라이브러리부터 설치할께요.
!pip install konlpy pandas seaborn gensim wordcloud

In [4]:
## import sklearn
import pandas as pd
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import tensorflow as tf
import nltk
from sklearn.linear_model import SGDClassifier
from konlpy.tag import Mecab
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
mecab = Mecab()
#fm.findSystemFonts()
plt.rcParams['font.family']= ["NanumGothicCoding"]
plt.rcParams["axes.unicode_minus"]=False
# GPU 환경 설정하기
# assert tf.test.is_gpu_available() == True, 'GPU 설정을 확인하세요.'
print(tf.config.list_physical_devices('GPU'))
print(tf.config.list_logical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[LogicalDevice(name='/device:GPU:0', device_type='GPU')]


### 1. 데이터 가져오기

#### * n-gram

In [25]:
# 2. data preprocessing에서 저장한 결과를 불러오고 shape을 확인합니다.
train_data = pd.read_csv('./train_data.csv')
val_data = pd.read_csv('./val_data.csv')

val_data.tokenized = val_data.tokenized.fillna('가')

val_data.isnull().sum()

#### * word Embedding

In [ ]:
# 2. data preprocessing에서 저장한 결과를 불러오고 shape을 확인합니다.
x_train = pd.read_csv('./x_train.csv')
x_train = x_train.values
x_val = pd.read_csv('./x_val.csv')
x_val = x_val.values
y_train = pd.read_csv('./y_train.csv')
y_val = pd.read_csv('./y_val.csv')

### 2. Machine Learning(N-grams)

#### 2-1. Naive Bayes

In [26]:
# N-grams으로 vectorize한 데이터를 Naive Bayes 모델로 학습합니다.
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import numpy as np

cv = CountVectorizer()
x_train_counts = cv.fit_transform(train_data.tokenized)
tf_transformer = TfidfTransformer(use_idf=False).fit(x_train_counts)
x_train_tf = tf_transformer.transform(x_train_counts)

clf = MultinomialNB().fit(x_train_tf, train_data.label)

text_clf = Pipeline([
      ('vect', CountVectorizer()),
      ('tfidf', TfidfTransformer()),
      ('clf', MultinomialNB()),
 ])

text_clf.fit(train_data.tokenized, train_data.label)

docs_test = val_data.tokenized
predicted = text_clf.predict(docs_test)
np.mean(predicted == val_data.label)

#### 2-2. 다른 모델들을 활용하여 학습합니다.

---

### 3. Deep Learning(Sequence)

In [27]:
# sequence 형태로 vectorize한 데이터를 딥러닝으로 학습합니다.
# Tip : 처음에는 embedding이 필요합니다
from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

embedding_dim = 100
hidden_units = 128
vocab_size = 11727

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(GRU(hidden_units))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train, epochs=10, callbacks=[es, mc], batch_size=64, validation_split=0.2)

In [ ]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(x_val, y_val)[1]))

#### 3-1. Conv1D

In [14]:
# Conv1D로 학습하고 모델 성능을 검증합니다.

#### 3-2. LSTM

In [ ]:
# LSTM으로 학습하고 모델 성능을 검증합니다.